In [28]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [29]:
import sys
import os
import h5py
import json
import scipy.io
import numpy as np
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation, concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Lambda, Conv2DTranspose
from keras.layers import Input
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json
from sklearn.preprocessing import StandardScaler

In [30]:
def Nuclei_feat(mag):
    nuc_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/nuc_Feature/'

    nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_' + mag + '.mat'
    nuc_data = []
    with h5py.File(nuc_file_name) as f:
        for column in f['feature']:
            row_data = []
            for row_number in range(len(column)):            
                row_data.append(f[column[row_number]][:])   
            nuc_data.append(row_data)
        
    nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_val_' + mag + '.mat'
    nuc_data_val = []
    with h5py.File(nuc_file_name) as f:
        for column in f['feature']:
            row_data = []
            for row_number in range(len(column)):            
                row_data.append(f[column[row_number]][:])   
            nuc_data_val.append(row_data)        

    nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_test_' + mag + '.mat'
    nuc_data_test = []
    with h5py.File(nuc_file_name) as f:
        for column in f['feature']:
            row_data = []
            for row_number in range(len(column)):            
                row_data.append(f[column[row_number]][:])   
            nuc_data_test.append(row_data)   
            
    Nuc_train_feat = np.array(nuc_data)
    Nuc_val_feat = np.array(nuc_data_val)
    Nuc_test_feat = np.array(nuc_data_test)
    del nuc_data, nuc_data_val, nuc_data_test

    Nuc_train_feat = Nuc_train_feat.reshape(len(Nuc_train_feat), 256,256,1)
    Nuc_val_feat = Nuc_val_feat.reshape(len(Nuc_val_feat), 256,256,1)
    Nuc_test_feat = Nuc_test_feat.reshape(len(Nuc_test_feat), 256,256,1)
    
    Nuc_train_feat_sc = nuc_scaler(Nuc_train_feat)
    Nuc_val_feat_sc = nuc_scaler(Nuc_val_feat)
    Nuc_test_feat_sc = nuc_scaler(Nuc_test_feat)
    
    return Nuc_train_feat_sc, Nuc_val_feat_sc, Nuc_test_feat_sc


In [31]:
def nuc_scaler(data):
    scaler = StandardScaler()
    data = data.reshape((len(data),256*256))
    scaler.fit(data)
    data_sc = scaler.transform(data)
    data_sc = data_sc.reshape((len(data),256, 256,1))
    return data_sc

In [32]:
def pca_scaler(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data_sc = scaler.transform(data)
    return data_sc

In [33]:
def PCA_feat(mag):
    pca_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/PCA_Feature/'

    pca_file_name = pca_dir+ 'breakhis_pca_feature_' + mag + '.mat'
    pca_data = []
    with h5py.File(pca_file_name) as f:
        for column in f['feature']:
            pca_data.append(column)
        
    pca_file_name = pca_dir+ 'breakhis_pca_feature_val_' + mag + '.mat'
    pca_data_val = []
    with h5py.File(pca_file_name) as f:
        for column in f['feature']:
            pca_data_val.append(column)
            
    pca_file_name = pca_dir+ 'breakhis_pca_feature_test_' + mag + '.mat'
    pca_data_test = []
    with h5py.File(pca_file_name) as f:
        for column in f['feature']:
            pca_data_test.append(column)
            
    PCA_train_data = np.array(pca_data)
    PCA_val_data = np.array(pca_data_val)
    PCA_test_data = np.array(pca_data_test)

    PCA_train_sc = pca_scaler(PCA_train_data)
    PCA_val_sc = pca_scaler(PCA_val_data)
    PCA_test_sc = pca_scaler(PCA_test_data)
    
    del PCA_train_data, PCA_val_data, PCA_test_data

    return PCA_train_sc, PCA_val_sc, PCA_test_sc

In [34]:
def mag_conn(mag):
    train_dir = 'breast_' + mag + '_split/train'
    val_dir = 'breast_' + mag + '_split/val'
    test_dir = 'breast_' + mag + '_split/test'

    if mag == '40':
        b_train, m_train, b_test, m_test, b_val, m_val = 500, 1170, 100, 137, 137, 174
    if mag == '100':
        b_train, m_train, b_test, m_test, b_val, m_val = 515, 1237, 100, 145, 135, 188
    if mag == '200':
        b_train, m_train, b_test, m_test, b_val, m_val = 498, 1190, 100, 139, 137, 178
    if mag == '400':
        b_train, m_train, b_test, m_test, b_val, m_val = 470, 1032, 100, 124, 140, 147        
    
    n_train =  b_train + m_train
    n_val =  b_val + m_val
    n_test =  b_test + m_test

    train_labels = np.array([0] * b_train + [1] * m_train)
    val_labels = np.array([0] * b_val + [1] * m_val)
    test_labels = np.array([0] * b_test + [1] * m_test)
    return train_dir, val_dir, test_dir, n_train, n_val, n_test, train_labels, val_labels, test_labels

In [35]:
def VGG_feat(mag):
    vgg_train = np.load('fine_bottleneck_feat_train_' + mag +'.npy')
    vgg_val = np.load('fine_bottleneck_feat_val_' + mag +'.npy')
    vgg_test = np.load('fine_bottleneck_feat_test_' + mag +'.npy')
    return vgg_train, vgg_val, vgg_test

In [36]:
def train_merge_model(mag):
    train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag)
    Nuc_train, Nuc_val, Nuc_test = Nuclei_feat(mag)
    PCA_train, PCA_val, PCA_test = PCA_feat(mag)
    VGG_train, VGG_val, VGG_test = VGG_feat(mag)

    model = merge_model(mag)
    model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
    
    model.fit(x = [Nuc_train, PCA_train, VGG_train], y = train_labels,
             validation_data = ([Nuc_val, PCA_val, VGG_val], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)
    
    test_score = model.evaluate(x = [Nuc_test, PCA_test, VGG_test], y = test_labels)
    print('\n', 'BreakHis Test accuracy:', test_score[1])

    model_new_name = 'Final_concat_Model_t2_' + mag
    model_json = model.to_json()
    with open(model_new_name + '.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(model_new_name + '.h5')
    return model

In [37]:
def merge_model(mag):
    ############################################################################################
    # Input 1 (Nuclei)
    ############################################################################################
    model_1 = Sequential()
    model_1.add(Flatten(input_shape= (256, 256, 1)))
    model_1.add(Dropout(0.5))
    model_1.add(Activation('relu'))
    #model_1.add(Dense(512, activation='sigmoid'))
    model_1.add(Dropout(0.5))
    model_1.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
    ############################################################################################
    # Input 2 (PCA)
    ############################################################################################
    PCA_input = Input(shape=(12,))
    model_2 = Sequential()
    model_2.add(Dense(512, input_dim=12))
    model_2.add(Activation('relu'))
    model_2.add(Dropout(0.5))
    #model_2.add(Dense(6))
    model_2.add(Activation('sigmoid'))
    model_2.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
    ############################################################################################
    # Input 3 (VGG)
    ############################################################################################
    model_3 = Sequential()
    model_3.add(Flatten(input_shape= (7, 7, 512)))
    model_3.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])

    merge_layer = concatenate([model_1.output, model_2.output, model_3.output])
    dnn_output = Dense(1, activation="sigmoid")(merge_layer)
    model_concat = Model(inputs=[model_1.input, model_2.input, model_3.input], outputs=dnn_output)
    return model_concat

In [38]:
def retrain(mag):
    train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag)
    Nuc_train, Nuc_val, Nuc_test = Nuclei_feat(mag)
    PCA_train, PCA_val, PCA_test = PCA_feat(mag)
    VGG_train, VGG_val, VGG_test = VGG_feat(mag)

    
    model_name = 'Final_concat_Model_t2_' + mag
    # load json and create model
    json_file = open(model_name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(model_name + '.h5')
    print("Loaded model from disk")
    # evaluate loaded model on test data
    model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(x = [Nuc_train, PCA_train, VGG_train], y = train_labels,
             validation_data = ([Nuc_val, PCA_val, VGG_val], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)
    
    test_score = model.evaluate(x = [Nuc_test, PCA_test, VGG_test], y = test_labels)
    print('\n', 'BreakHis Test accuracy:', test_score[1])

    model_name = 'Final_concat_Model_t2_' + mag
    model_json = model.to_json()
    with open(model_name + '.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(model_name + '.h5')
    return model

In [39]:
batch_size = 5
epochs = 20
mag_fact = '100'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1752 samples, validate on 323 samples
Epoch 1/20
1752/1752 [==============================] - 4s 2ms/step - loss: 1.9287 - accuracy: 0.5902 - val_loss: 1.6275 - val_accuracy: 0.6316
Epoch 2/20
1752/1752 [==============================] - 4s 2ms/step - loss: 1.5835 - accuracy: 0.6849 - val_loss: 1.6565 - val_accuracy: 0.6347
Epoch 3/20
1752/1752 [==============================] - 4s 2ms/step - loss: 1.2792 - accuracy: 0.7477 - val_loss: 3.5603 - val_accuracy: 0.5975
Epoch 4/20
1752/1752 [==============================] - 4s 2ms/step - loss: 1.0613 - accuracy: 0.7968 - val_loss: 2.2434 - val_accuracy: 0.6842
Epoch 5/20
1752/1752 [==============================] - 4s 2ms/step - loss: 0.9414 - accuracy: 0.7991 - val_loss: 1.9339 - val_accuracy: 0.7214
Epoch 6/20
1752/1752 [==============================] - 4s 2ms/step - loss: 0.9022 - accuracy: 0.8282 - val_loss: 2.0201 - val_accuracy: 0.7585
Epoch 7/20
1752/1752 [==============================] - 4s 2ms/step - loss: 0.7693 - accu

In [40]:
batch_size = 20
epochs = 400
mag_fact = '100'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

Loaded model from disk
Train on 1752 samples, validate on 323 samples
Epoch 1/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.8264 - accuracy: 0.8830 - val_loss: 3.2184 - val_accuracy: 0.7430
Epoch 2/400
1752/1752 [==============================] - 3s 2ms/step - loss: 1.1012 - accuracy: 0.8584 - val_loss: 3.9721 - val_accuracy: 0.7245
Epoch 3/400
1752/1752 [==============================] - 3s 2ms/step - loss: 1.2977 - accuracy: 0.8619 - val_loss: 5.6076 - val_accuracy: 0.6687
Epoch 4/400
1752/1752 [==============================] - 3s 2ms/step - loss: 1.1345 - accuracy: 0.8676 - val_loss: 6.5838 - val_accuracy: 0.6687
Epoch 5/400
1752/1752 [==============================] - 3s 2ms/step - loss: 1.2312 - accuracy: 0.8739 - val_loss: 5.6352 - val_accuracy: 0.6966
Epoch 6/400
1752/1752 [==============================] - 3s 2ms/step - loss: 1.1304 - accuracy: 0.8773 - val_loss: 5.3776 - val_accuracy: 0.7245
Epoch 7/400
1752/1752 [==============================] - 3s 

In [41]:
batch_size = 32
epochs = 400
mag_fact = '100'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

Loaded model from disk
Train on 1752 samples, validate on 323 samples
Epoch 1/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.3401 - accuracy: 0.9846 - val_loss: 16.4571 - val_accuracy: 0.7740
Epoch 2/400
1752/1752 [==============================] - 3s 2ms/step - loss: 0.4028 - accuracy: 0.9760 - val_loss: 15.6264 - val_accuracy: 0.7802
Epoch 3/400
1752/1752 [==============================] - 3s 2ms/step - loss: 0.4046 - accuracy: 0.9795 - val_loss: 16.0593 - val_accuracy: 0.7802
Epoch 4/400
1752/1752 [==============================] - 3s 2ms/step - loss: 0.2807 - accuracy: 0.9806 - val_loss: 16.1876 - val_accuracy: 0.7740
Epoch 5/400
1752/1752 [==============================] - 3s 2ms/step - loss: 0.1558 - accuracy: 0.9897 - val_loss: 16.2632 - val_accuracy: 0.7740
Epoch 6/400
1752/1752 [==============================] - 3s 2ms/step - loss: 0.2642 - accuracy: 0.9874 - val_loss: 16.5471 - val_accuracy: 0.7678
Epoch 7/400
1752/1752 [==============================]

In [42]:
batch_size = 10
epochs = 400
mag_fact = '100'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

Loaded model from disk
Train on 1752 samples, validate on 323 samples
Epoch 1/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.2453 - accuracy: 0.9863 - val_loss: 16.7980 - val_accuracy: 0.7647
Epoch 2/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.1506 - accuracy: 0.9880 - val_loss: 16.9343 - val_accuracy: 0.7740
Epoch 3/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.2235 - accuracy: 0.9852 - val_loss: 15.9428 - val_accuracy: 0.7647
Epoch 4/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.2325 - accuracy: 0.9857 - val_loss: 18.7209 - val_accuracy: 0.7616
Epoch 5/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.2123 - accuracy: 0.9880 - val_loss: 17.1506 - val_accuracy: 0.7678
Epoch 6/400
1752/1752 [==============================] - 4s 2ms/step - loss: 0.2946 - accuracy: 0.9834 - val_loss: 16.6615 - val_accuracy: 0.7585
Epoch 7/400
1752/1752 [==============================]

In [43]:
batch_size = 50
epochs = 1000
mag_fact = '100'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

Loaded model from disk
Train on 1752 samples, validate on 323 samples
Epoch 1/1000
1752/1752 [==============================] - 4s 2ms/step - loss: 0.3336 - accuracy: 0.9863 - val_loss: 29.6719 - val_accuracy: 0.7647
Epoch 2/1000
1752/1752 [==============================] - 3s 2ms/step - loss: 0.6051 - accuracy: 0.9783 - val_loss: 29.4872 - val_accuracy: 0.7647
Epoch 3/1000
1752/1752 [==============================] - 3s 2ms/step - loss: 0.3856 - accuracy: 0.9846 - val_loss: 28.9504 - val_accuracy: 0.7678
Epoch 4/1000
1752/1752 [==============================] - 4s 2ms/step - loss: 0.6099 - accuracy: 0.9840 - val_loss: 29.0459 - val_accuracy: 0.7678
Epoch 5/1000
1752/1752 [==============================] - 3s 2ms/step - loss: 0.4345 - accuracy: 0.9840 - val_loss: 29.0308 - val_accuracy: 0.7709
Epoch 6/1000
1752/1752 [==============================] - 3s 2ms/step - loss: 0.5570 - accuracy: 0.9823 - val_loss: 29.0088 - val_accuracy: 0.7709
Epoch 7/1000
1752/1752 [========================

In [44]:
batch_size = 1
epochs = 20
mag_fact = '100'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

Loaded model from disk
Train on 1752 samples, validate on 323 samples
Epoch 1/20
1752/1752 [==============================] - 10s 6ms/step - loss: 8.9826 - accuracy: 0.9018 - val_loss: 47.0277 - val_accuracy: 0.7368
Epoch 2/20
1752/1752 [==============================] - 10s 6ms/step - loss: 23.4816 - accuracy: 0.8282 - val_loss: 71.7683 - val_accuracy: 0.6997
Epoch 3/20
1752/1752 [==============================] - 9s 5ms/step - loss: 28.4774 - accuracy: 0.8328 - val_loss: 93.7037 - val_accuracy: 0.6873
Epoch 4/20
1752/1752 [==============================] - 9s 5ms/step - loss: 32.4758 - accuracy: 0.8311 - val_loss: 60.8088 - val_accuracy: 0.7152
Epoch 5/20
1752/1752 [==============================] - 10s 6ms/step - loss: 31.2437 - accuracy: 0.8356 - val_loss: 73.5690 - val_accuracy: 0.7368
Epoch 6/20
1752/1752 [==============================] - 10s 6ms/step - loss: 31.9272 - accuracy: 0.8476 - val_loss: 73.5195 - val_accuracy: 0.7523
Epoch 7/20
1752/1752 [=============================

In [45]:
batch_size = 5
epochs = 20
mag_fact = '40'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

batch_size = 5
epochs = 20
mag_fact = '200'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

batch_size = 5
epochs = 20
mag_fact = '400'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1670 samples, validate on 311 samples
Epoch 1/20
1670/1670 [==============================] - 3s 2ms/step - loss: 1.9103 - accuracy: 0.6048 - val_loss: 2.3679 - val_accuracy: 0.5981
Epoch 2/20
1670/1670 [==============================] - 2s 1ms/step - loss: 1.4275 - accuracy: 0.7036 - val_loss: 1.7404 - val_accuracy: 0.6527
Epoch 3/20
1670/1670 [==============================] - 2s 1ms/step - loss: 1.2702 - accuracy: 0.7383 - val_loss: 2.4585 - val_accuracy: 0.6431
Epoch 4/20
1670/1670 [==============================] - 2s 1ms/step - loss: 1.0211 - accuracy: 0.7904 - val_loss: 2.4128 - val_accuracy: 0.6527
Epoch 5/20
1670/1670 [==============================] - 2s 1ms/step - loss: 0.8727 - accuracy: 0.8060 - val_loss: 1.6496 - val_accuracy: 0.6881
Epoch 6/20
1670/1670 [==============================] - 2s 1ms/step - loss: 0.8101 - accuracy: 0.8228 - val_loss: 2.1628 - val_accuracy: 0.6720
Epoch 7/20
1670/1670 [==============================] - 2s 1ms/step - loss: 0.7002 - accu

In [46]:
batch_size = 5
epochs = 400
mag_fact = '40'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)


batch_size = 5
epochs = 400
mag_fact = '200'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

batch_size = 5
epochs = 400
mag_fact = '400'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

Loaded model from disk
Train on 1670 samples, validate on 311 samples
Epoch 1/400
1670/1670 [==============================] - 3s 2ms/step - loss: 10.0374 - accuracy: 0.6922 - val_loss: 21.4424 - val_accuracy: 0.6013
Epoch 2/400
1670/1670 [==============================] - 2s 1ms/step - loss: 12.1465 - accuracy: 0.7210 - val_loss: 22.2517 - val_accuracy: 0.6559
Epoch 3/400
1670/1670 [==============================] - 2s 1ms/step - loss: 9.7813 - accuracy: 0.7641 - val_loss: 24.1772 - val_accuracy: 0.6431
Epoch 4/400
1670/1670 [==============================] - 2s 1ms/step - loss: 9.7956 - accuracy: 0.7838 - val_loss: 16.8299 - val_accuracy: 0.6881
Epoch 5/400
1670/1670 [==============================] - 2s 1ms/step - loss: 9.0093 - accuracy: 0.8018 - val_loss: 21.3416 - val_accuracy: 0.6656
Epoch 6/400
1670/1670 [==============================] - 2s 1ms/step - loss: 7.7613 - accuracy: 0.8287 - val_loss: 25.0338 - val_accuracy: 0.6849
Epoch 7/400
1670/1670 [=============================